<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #3: Upload clips to Zooniverse</h1>
<h3 align="right"><a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso/blob/master/tutorials/03_Upload_clips_to_Zooniverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h3>
<h3 align="right">Written by KSO Team</h3>

This notebook takes you through the process of:

* Connecting to a Zooniverse project

* Extracting clips from videos stored locally or in the cloud.

* Modify these clips with for example a color correction or video compression.

* Upload the clips to Zooniverse for the 1st part of the species identification workflow (when does X species appear on a video).

If you do not have a project with us yet, you can run the template project to get a taste of how it all works. Only the uploading of the clips to Zooniverse will not be possible if you are not a member of our template project on Zooniverse.

🔴 <span style="color:red">&nbsp;NOTE: If you want to run another project than the template project, you need to have a Zooniverse account and be a member of the corresponding project.  </span>

# Set up KSO requirements

### Install all the requirements

If you are running this notebook in Google Colab, this cell should take ~2 mins and might restart the session. Please run this cell until you get the "Successful installation!" message.

In [ ]:
import sys
import os

# Check if notebook is running in colab
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # Clone kso repo and install requirements
    if not os.path.exists("kso"):
        print("Installing all dependencies...")
        !git clone https://github.com/ocean-data-factory-sweden/kso.git
        !pip install \
        git+https://github.com/zooniverse/panoptes-python-client.git \
        ipysheet \
        jupyter_bbox_widget \
        av \
        boto3 \
        dataclass_csv \
        fiftyone \
        ffmpeg \
        csv_diff \
        wandb \
        mlflow \
        lida \
        pims

    # Restart colab session if conflicting libraries can't be imported
    try:
        import panoptes_client
    except ImportError:
        os.kill(os.getpid(), 9)

    # Enable external widgets and navigate to the kso tutorial folder
    try:
        from google.colab import output

        output.enable_custom_widget_manager()
        os.chdir("kso/tutorials")
    except ImportError:
        pass

# Prepare the dev settings if needed
try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
        import kso_utils

        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except ImportError:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Avoid issues with widgets not displaying properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

# Load the clear output function to keep things clean
from IPython.display import clear_output

clear_output()
print("Successful installation... you're good to go!")

### Import Python packages

In [ ]:
# Import required modules for tut#3
import kso_utils.widgets as kso_widgets
import kso_utils.project_utils as p_utils
import kso_utils.tutorials_utils as t_utils
from kso_utils.project import ProjectProcessor

print("Packages loaded successfully")

### Specify GPU availability

In [ ]:
gpu_available = kso_widgets.gpu_select()

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

### Initiate project's database

In [ ]:
# Save the name of the project
project = p_utils.find_project(project_name=project_name.value)

# Initiate pp
pp = ProjectProcessor(project)

### Specify to request (or not) the latest Zooniverse info

In [ ]:
latest_zoo_info = kso_widgets.request_latest_zoo_info()

### Connect and retrieve information from the Zooniverse project

In [ ]:
pp.connect_zoo_project(latest_zoo_info.result)

# Select the movie to upload to Zooniverse

### Check what movies are available

In [ ]:
pp.get_movie_info()

### Specify movie of interest

In [ ]:
pp.preview_media()

### Check if movie is already in Zooniverse

In [ ]:
# Remember to query the newest zooniverse data to get the most up to date list of clips uploaded
pp.check_movies_uploaded(pp.movie_selected)

# Create some clip examples (Optional)
Test different parameters (e.g. compression rate, color modifications) in randomly selected clip examples

In [ ]:
pp.generate_zoo_clips(
    movie_name=pp.movie_selected,
    movie_path=pp.movie_path,
    is_example=True,
    use_gpu=gpu_available.result,
)

In [ ]:
t_utils.check_clip_size(clips_list=pp.generated_clips.clip_path)

In [ ]:
kso_widgets.compare_clips(
    example_clips=pp.generated_clips.clip_path,
    modified_clips=pp.generated_clips.modif_clip_path,
)

# Create the clips to upload to Zooniverse

In [ ]:
pp.generate_zoo_clips(
    movie_name=pp.movie_selected,
    movie_path=pp.movie_path,
    is_example=False,
    use_gpu=gpu_available.result,
)

### Check the size of the clips

In [ ]:
t_utils.check_clip_size(clips_list=pp.generated_clips.clip_path)

# Upload clips to Zooniverse

If you receive an error message related to file size, we recommend shortening the clip length or compressing the clips to achieve a suitable filesize.

Make sure your workflows in Zooniverse have different names to avoid issues while creating a new workflow

🔴 <span style="color:red">&nbsp;NOTE: If you run the template project without being a member of our template project, it is not possible to run this last cell.  </span>

In [ ]:
pp.upload_zoo_subjects("clip")

In [ ]:
# END